In [2]:
import pandas as pd
import numpy as np
import scipy.stats as st

## Data

In [3]:
path = ""

In [4]:
dataset = pd.read_csv(path+"nba_games_2013_2015.csv",sep = ";")
dataset.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22015,1610612750,MIN,Minnesota Timberwolves,21501226,2016-04-13,MIN vs. NOP,W,240,144,...,0.826,5,38,43,41,14,8,13,20,35.0
1,22015,1610612749,MIL,Milwaukee Bucks,21501225,2016-04-13,MIL vs. IND,L,240,92,...,0.846,7,36,43,23,8,3,15,15,-5.0
2,22015,1610612738,BOS,Boston Celtics,21501217,2016-04-13,BOS vs. MIA,W,240,98,...,0.864,10,29,39,20,7,3,7,20,10.0
3,22015,1610612747,LAL,Los Angeles Lakers,21501228,2016-04-13,LAL vs. UTA,W,239,101,...,0.867,8,39,47,19,6,3,13,17,5.0
4,22015,1610612739,CLE,Cleveland Cavaliers,21501220,2016-04-13,CLE vs. DET,L,265,110,...,0.733,8,35,43,21,4,7,10,23,-2.0


In [5]:
len(dataset)

7380

### Split the data into 3 separate files according to the season!

In [13]:
dataset_2014 = dataset[dataset.SEASON_ID == 22013]
dataset_2015 = dataset[dataset.SEASON_ID == 22014]
dataset_2016 = dataset[dataset.SEASON_ID == 22015]

### Test, if the hypothesis that offensive production of Cleveland Cavaliers and Golden State Warriors (teams from finals) was distributed equally in 2015/2016

In [15]:
# PTS
# FG_PCT

In [16]:
cle_pts = dataset_2016[dataset_2016.TEAM_ABBREVIATION == "CLE"].PTS.values
gsw_pts = dataset_2016[dataset_2016.TEAM_ABBREVIATION == "GSW"].PTS.values

In [17]:
# t 2 sample test
st.ttest_ind(cle_pts,gsw_pts)

Ttest_indResult(statistic=-5.973920595717936, pvalue=1.4233420547764935e-08)

In [19]:
cle_FG_PCT = dataset_2016[dataset_2016.TEAM_ABBREVIATION == "CLE"].FG_PCT.values
gsw_FG_PCT = dataset_2016[dataset_2016.TEAM_ABBREVIATION == "GSW"].FG_PCT.values

In [20]:
# t 2 sample test
st.ttest_ind(cle_FG_PCT,gsw_FG_PCT)

Ttest_indResult(statistic=-3.13195297595825, pvalue=0.00206097581047554)

### Test whether production of Cleveland changes significantly before and after coach change in 2015/2016 - Use ony data from seasons 2014/2015 and 2015/2016 - Those are when cleveland was coached by Blatt
#### Coach Blatt was fired on 24th of Jan

In [ ]:
# 1st solution
# take last x games before and first x games after and try t-test

In [30]:
cle_dataset = dataset[(dataset.TEAM_ABBREVIATION == "CLE") & (dataset.SEASON_ID.isin([22014,22015]))][["GAME_DATE","WL","PTS"]]

In [31]:
first_30_after = cle_dataset[cle_dataset.GAME_DATE >= '2016-01-24'].sort_values("GAME_DATE").head(30)["PTS"].values
last_30_before = cle_dataset[cle_dataset.GAME_DATE < '2016-01-24'].sort_values("GAME_DATE", ascending = False).head(30)["PTS"].values

In [32]:
# t 2 sample test
st.ttest_ind(first_30_after,last_30_before)

Ttest_indResult(statistic=2.683092633471887, pvalue=0.009489092058459141)

In [ ]:
# 2nd solution
# kruskal test, samples can have different sizes

In [33]:
all_after = cle_dataset[cle_dataset.GAME_DATE >= '2016-01-24'].sort_values("GAME_DATE")["PTS"].values
all_before = cle_dataset[cle_dataset.GAME_DATE < '2016-01-24'].sort_values("GAME_DATE", ascending = False)["PTS"].values

In [34]:
st.kruskal(all_after,all_before)

KruskalResult(statistic=7.961718139705661, pvalue=0.004777703982670637)

### Download the same dataset for playoffs games in 2016

In [40]:
df_playoffs = pd.read_csv(path+"nba_playoff_games_2016.csv",sep = ";")
df_playoffs.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,42015,1610612739,CLE,Cleveland Cavaliers,41500407,2016-06-19,CLE @ GSW,W,241,93,...,0.840,9,39,48,17,7,6,11,15,4.0
1,42015,1610612744,GSW,Golden State Warriors,41500407,2016-06-19,GSW vs. CLE,L,239,89,...,0.769,7,32,39,22,7,5,10,23,-4.0
2,42015,1610612744,GSW,Golden State Warriors,41500406,2016-06-16,GSW @ CLE,L,238,101,...,0.690,9,26,35,19,5,3,14,25,-14.0
3,42015,1610612739,CLE,Cleveland Cavaliers,41500406,2016-06-16,CLE vs. GSW,W,240,115,...,0.781,8,37,45,24,12,7,10,25,14.0
4,42015,1610612739,CLE,Cleveland Cavaliers,41500405,2016-06-13,CLE @ GSW,W,241,112,...,0.609,8,33,41,15,11,9,16,22,15.0


### Select Toronto Raptors and test the hypothesis that number of blocks in playoffs are from the same distribution as in reguar season 2015/2016

In [41]:
reg_season = dataset_2016[(dataset_2016.TEAM_NAME == "Toronto Raptors")]["BLK"].values
playoffs = df_playoffs[(df_playoffs.TEAM_NAME == "Toronto Raptors")]["BLK"].values

In [42]:
st.kruskal(reg_season,playoffs)

KruskalResult(statistic=7.483641456983013, pvalue=0.006226202691250736)

### Test the hypothesis that points per game are equally distributed in all 3 seasons for Cleveland

In [43]:
dataset_CLE = dataset[dataset.TEAM_ABBREVIATION == "CLE"]

In [44]:
cle_2014 = dataset_CLE[dataset_CLE.SEASON_ID == 22013].PTS.values
cle_2015 = dataset_CLE[dataset_CLE.SEASON_ID == 22014].PTS.values
cle_2016 = dataset_CLE[dataset_CLE.SEASON_ID == 22015].PTS.values

In [45]:
st.f_oneway(cle_2014,cle_2015,cle_2016)

F_onewayResult(statistic=5.9200250318080885, pvalue=0.003087727119983984)

#### Between which seasons, we can see the significant difference?
+ unfortunatelly, this is not the output of ANOVA test and further analysis needs to be applied in most of the cases
+ Note that Lebron James came back to Cleveland prior to season 2014/2015 (just for interpretation of the results)

In [46]:
st.ttest_ind(cle_2014,cle_2015)

Ttest_indResult(statistic=-2.508958204796911, pvalue=0.013091680534336523)

In [47]:
st.ttest_ind(cle_2015,cle_2016)

Ttest_indResult(statistic=-0.6442093460555935, pvalue=0.5203507617734474)

In [48]:
st.ttest_ind(cle_2014,cle_2016)

Ttest_indResult(statistic=-3.339057501969076, pvalue=0.001043164899206325)

 We can see that there are significant differences in distributions between 2014 and 2015, as well as 2014 and 2016. On the other hand 2015 and 2016 are from the same distribution. Lebron James came back to Cleveland at the end of 2014 so they improved their scoring for the next two seasons.